### Disclaimer

Distribution authorized to U.S. Government agencies and their contractors. Other requests for this document shall be referred to the MIT Lincoln Laboratory Technology Office.

This material is based upon work supported by the Under Secretary of Defense for Research and Engineering under Air Force Contract No. FA8702-15-D-0001. Any opinions, findings, conclusions or recommendations expressed in this material are those of the author(s) and do not necessarily reflect the views of the Under Secretary of Defense for Research and Engineering.

© 2019 Massachusetts Institute of Technology.

The software/firmware is provided to you on an As-Is basis

Delivered to the U.S. Government with Unlimited Rights, as defined in DFARS Part 252.227-7013 or 7014 (Feb 2014). Notwithstanding any copyright notice, U.S. Government rights in this work are defined by DFARS 252.227-7013 or DFARS 252.227-7014 as detailed above. Use of this work other than as specifically authorized by the U.S. Government may violate any copyrights that exist in this work.


In [1]:
from tesse.env import Env
from tesse.msgs import *

import time
import matplotlib.pyplot as plt
import defusedxml.ElementTree as ET
import numpy as np

In [2]:
env = Env()

### Demo Transform Message

In [ ]:
env.send(Transform(-1, 0, 0)) # left
time.sleep(1)
env.send(Transform(1, 0, 0)) # right
time.sleep(1)
env.send(Transform(0, -1, 0)) # back
time.sleep(1)
env.send(Transform(0, 1, 0)) # forward
time.sleep(1)
env.send(Transform(0, 0, 45)) # turn right
time.sleep(1)
env.send(Transform(0, 0, -45)) # turn left

### Demo Force and Torque Message

In [ ]:
env.send(AddForce(100, 0)) # Forward force
time.sleep(1)
env.send(AddForce(-100, 0)) # Backward force
time.sleep(1)
env.send(AddForce(0, 10)) # Torque right
time.sleep(1)
env.send(AddForce(0, -10)) # Torque left

### Demo Reposition Message

In [ ]:
env.send(Reposition(-18, 14, 24, 0, 0, 0, 0))
time.sleep(1)
env.send(Reposition(-17, 2, 25, 0, 0, 0, 0))

### Demo getting images

In [ ]:
def plot(response, title=None):
    if len(response.images) == 0:
        return
    figsize = (len(response.images)*5, 5)
    fig, axs = plt.subplots(1, len(response.images), figsize=figsize)
    if len(response.images) > 1:
        for ax, img in zip(axs, response.images):
            ax.axis('off');
            ax.imshow(img, cmap='gray');
    else:
        axs.axis('off');
        axs.imshow(response.images[0], cmap='gray');
        axs.set_title(title)

In [ ]:
response = env.request(DataRequest())
if response is not None:
    print(response.metadata)
    plot(response)

In [ ]:
cameras=[
    (Camera.RGB_LEFT, Compression.OFF, Channels.THREE),
    (Camera.SEGMENTATION, Compression.OFF, Channels.THREE),
]
response = env.request(DataRequest(metadata=False, cameras=cameras))
if response is not None:
    plot(response)

### Metadata request

In [43]:
response = env.request(MetadataRequest())
if response is not None:
    print(response.metadata)

<TESSE_Agent_Metadata_v0.4>
  <position x='29.85107' y='2.499892' z='36.96291'/>
  <quaternion x='0' y='0.5910133' z='0' w='0.8066618'/>
  <velocity x_dot='0' y_dot='3.080545E-05' z_dot='0'/>
  <angular_velocity x_ang_dot='0' y_ang_dot='0' z_ang_dot='0'/>
  <acceleration x_ddot='0' y_ddot='6.255863E-05' z_ddot='0'/>
  <angular_acceleration x_ang_ddot='0' y_ang_ddot='0' z_ang_ddot='0'/>
  <time>165.7</time>
  <collision status='false' name=''/>
  <collider status='true'/>
</TESSE_Agent_Metadata_v0.4>



### Camera request

In [ ]:
response = env.request(CameraInformationRequest())
if response is not None:
    print(response.metadata)

### Demonstrate some camera manipulation

In [ ]:
# Reset 3rd person view to default
_ = env.request(SetCameraParametersRequest(768, 1024, 90, Camera.THIRD_PERSON))
_ = env.request(SetCameraPositionRequest(0, 1.319, -2.258, Camera.THIRD_PERSON))
response = env.request(DataRequest(cameras=[(Camera.THIRD_PERSON, Compression.OFF, Channels.THREE)]))
if response is not None:
    plot(response, 'default')

# Change the parameters and get the camera
_ = env.request(SetCameraParametersRequest(768, 1024, 55, Camera.THIRD_PERSON))
response = env.request(DataRequest(cameras=[(Camera.THIRD_PERSON, Compression.OFF, Channels.THREE)]))
if response is not None:
    plot(response, 'change fov')


# Change the position and get the camera
_ = env.request(SetCameraPositionRequest(0, 3, -5.5, Camera.THIRD_PERSON))
response = env.request(DataRequest(cameras=[(Camera.THIRD_PERSON, Compression.OFF, Channels.THREE)]))
if response is not None:
    plot(response, 'change position')

### Demo respawning

In [ ]:
for i in range(3):
    env.send(Respawn())
    response = env.request(DataRequest(cameras=[(Camera.THIRD_PERSON, Compression.OFF, Channels.THREE)]))
    if response is not None:
        plot(response)
    time.sleep(1)

### Demo changing scenes

*NOTE: loading scenes for the first time may take a long time*

In [50]:
scene_index = 3

response = env.request(SceneRequest(scene_index))
if response is not None:
    print(response.metadata)

<current_scene>
  <index>3</index>
  <name>tesse_multiagent_test_scene</name>
</current_scene>



### Demo step mode

This puts the game into step mode. The rate defines the update rate (Hz) for the game.

Then, we step forward several times showing how Unit time progresses.

In [ ]:
rate = 1
env.send(SetFrameRate(rate))

In [ ]:
def unity_time(env):
    response = env.request(MetadataRequest())
    root = ET.fromstring(response.metadata)
    return float(root.find('time').text)

In [ ]:
unity_start = unity_time(env)

for _ in range(10):
    env.send(StepWithForce())
    print("Elapsed time is ", unity_time(env) - unity_start, " seconds.")

In [ ]:
# Exit step mode by setting rate to 0
rate = 0
env.send(SetFrameRate(rate))

### Demo colliders

Turn collisions off, then back on

In [ ]:
env.send(ColliderRequest(enable=0)) # turn collisions off

In [ ]:
env.send(ColliderRequest(enable=1)) # turn collisions on

### Demo spawning and removing objects

In [63]:
x = 30
y = 2
z = 37
radius = 2
orientation = [ 0.4619398, 0.1913417, 0.4619398, 0.7325378 ]

env.send(Reposition(x, y, z, 0, 0, 0, 0))
time.sleep(.2)

for angle in range(0, 360, 60):
    response = env.request(SpawnObjectRequest(ObjectType.CUBE,
                                ObjectSpawnMethod.USER,
                                x + radius*np.cos(angle*np.pi/180), 
                                y, 
                                z + radius*np.sin(angle*np.pi/180),
                                *orientation)
            )
    time.sleep(.2)

In [64]:
env.request(RemoveObjectsRequest())

In [61]:
print(env.request(ObjectsRequest()).metadata)

<objects>
  <object>
    <type>CUBE</type>
    <position x ='32' y='2' z='37'/>
    <quaternion x ='0.4619398' y='0.1913417' z='0.4619398' w='0.7325378'/>
  </object>
  <object>
    <type>CUBE</type>
    <position x ='31' y='2' z='38.73205'/>
    <quaternion x ='0.4619398' y='0.1913417' z='0.4619398' w='0.7325378'/>
  </object>
  <object>
    <type>CUBE</type>
    <position x ='29' y='2' z='38.73205'/>
    <quaternion x ='0.4619398' y='0.1913417' z='0.4619398' w='0.7325378'/>
  </object>
  <object>
    <type>CUBE</type>
    <position x ='28' y='2' z='37'/>
    <quaternion x ='0.4619398' y='0.1913417' z='0.4619398' w='0.7325378'/>
  </object>
  <object>
    <type>CUBE</type>
    <position x ='29' y='2' z='35.26795'/>
    <quaternion x ='0.4619398' y='0.1913417' z='0.4619398' w='0.7325378'/>
  </object>
  <object>
    <type>CUBE</type>
    <position x ='31' y='2' z='35.26795'/>
    <quaternion x ='0.4619398' y='0.1913417' z='0.4619398' w='0.7325378'/>
  </object>
</objects>

